<a href="https://colab.research.google.com/github/NirbhayArora1209/NirbhayArora1209/blob/main/ML_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Football League Winner Prediction**

In [ ]:
#!python -m pip uninstall matplotlib # imp uncomment if matplotlib gives error after installing restart and comment again
#!pip install matplotlib==3.1.3
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
!pip install -q gwpy


     |████████████████████████████████| 1.4 MB 5.5 MB/s 
     |████████████████████████████████| 51 kB 5.8 MB/s 
     |████████████████████████████████| 11.2 MB 44.8 MB/s 
     |████████████████████████████████| 45 kB 2.7 MB/s 
     |████████████████████████████████| 4.1 MB 40.5 MB/s 
     |████████████████████████████████| 957 kB 39.6 MB/s 


In [ ]:
%%capture
base_url = 'https://understat.com/league'
leagues = ['La_liga', 'EPL', 'Bundesliga', 'Serie_A', 'Ligue_1', 'RFPL']
seasons = ['2014', '2015', '2016', '2017', '2018', '2019', '2020','2021','2022']

full_data = dict()
for league in leagues:
  
  season_data = dict()
  for season in seasons:    
    url = base_url+'/'+league+'/'+season
    res = requests.get(url,verify=False)
    soup = BeautifulSoup(res.content, "lxml")

    scripts = soup.find_all('script')
    
    string_with_json_obj = ''

    
    for el in scripts:
        if 'teamsData' in el.text:
          string_with_json_obj = el.text.strip()

  

    
    ind_start = string_with_json_obj.index("('")+2
    ind_end = string_with_json_obj.index("')")
    json_data = string_with_json_obj[ind_start:ind_end]
    json_data = json_data.encode('utf8').decode('unicode_escape')
    
    
  
    data = json.loads(json_data)
    
   
    teams = {}
    for id in data.keys():
      teams[id] = data[id]['title']
      

    columns = []
    
    values = []
    for id in data.keys():
      columns = list(data[id]['history'][0].keys())
      values = list(data[id]['history'][0].values())
      break
      
    
    dataframes = {}
    for id, team in teams.items():
      teams_data = []
      for row in data[id]['history']:
        teams_data.append(list(row.values()))

      df = pd.DataFrame(teams_data, columns=columns)
      dataframes[team] = df
     
      
    
    for team, df in dataframes.items():
        dataframes[team]['ppda_coef'] = dataframes[team]['ppda'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
        dataframes[team]['ppda_att'] = dataframes[team]['ppda'].apply(lambda x: x['att'])
        dataframes[team]['ppda_def'] = dataframes[team]['ppda'].apply(lambda x: x['def'])
        dataframes[team]['oppda_coef'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att']/x['def'] if x['def'] != 0 else 0)
        dataframes[team]['oppda_att'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['att'])
        dataframes[team]['oppda_def'] = dataframes[team]['ppda_allowed'].apply(lambda x: x['def'])
    
    frames = []
    for team, df in dataframes.items():
        df['team'] = team
        frames.append(df)
    
    full_stat = pd.concat(frames)
    full_stat = full_stat.drop(['ppda', 'ppda_allowed'], axis=1)
    
    full_stat['xG_diff'] = full_stat['xG'] - full_stat['scored']
    full_stat['xGA_diff'] = full_stat['xGA'] - full_stat['missed']
    full_stat['xpts_diff'] = full_stat['xpts'] - full_stat['pts']
    
    full_stat.reset_index(inplace=True, drop=True)
    season_data[season] = full_stat
  
  df_season = pd.concat(season_data)
  full_data[league] = df_season
  
data = pd.concat(full_data)
data.head()
data.index = data.index.droplevel(2)
data.index = data.index.rename(names=['league','year'], level=[0,1])
data.to_csv('understat_per_game.csv')

In [ ]:
df = pd.read_csv('/content/understat_per_game.csv')

In [ ]:
df.head()

,league,year,h_a,xG,xGA,npxG,npxGA,deep,deep_allowed,scored,...,ppda_coef,ppda_att,ppda_def,oppda_coef,oppda_att,oppda_def,team,xG_diff,xGA_diff,xpts_diff
0,La_liga,2014,h,1.321070,1.141510,0.438073,1.141510,4,5,1,...,12.071429,338,28,6.300000,189,30,Malaga,0.321070,1.141510,-1.4697
1,La_liga,2014,a,1.131960,1.968630,1.131960,1.069640,4,5,0,...,15.368421,292,19,6.066667,182,30,Malaga,1.131960,-1.031370,0.7086
2,La_liga,2014,h,0.819725,0.358265,0.819725,0.358265,7,1,0,...,9.846154,128,13,8.925926,241,27,Malaga,0.819725,0.358265,0.7526
3,La_liga,2014,a,0.446347,2.761880,0.446347,2.761880,6,2,2,...,5.500000,99,18,8.000000,264,33,Malaga,-1.553653,0.761880,-0.9372
4,La_liga,2014,h,0.278657,0.646364,0.278657,0.646364,4,7,0,...,9.451613,293,31,4.571429,96,21,Malaga,0.278657,0.646364,-0.1279


In [ ]:
df

,league,year,h_a,xG,xGA,npxG,npxGA,deep,deep_allowed,scored,...,ppda_coef,ppda_att,ppda_def,oppda_coef,oppda_att,oppda_def,team,xG_diff,xGA_diff,xpts_diff
0,La_liga,2014,h,1.321070,1.141510,0.438073,1.141510,4,5,1,...,12.071429,338,28,6.300000,189,30,Malaga,0.321070,1.141510,-1.4697
1,La_liga,2014,a,1.131960,1.968630,1.131960,1.069640,4,5,0,...,15.368421,292,19,6.066667,182,30,Malaga,1.131960,-1.031370,0.7086
2,La_liga,2014,h,0.819725,0.358265,0.819725,0.358265,7,1,0,...,9.846154,128,13,8.925926,241,27,Malaga,0.819725,0.358265,0.7526
3,La_liga,2014,a,0.446347,2.761880,0.446347,2.761880,6,2,2,...,5.500000,99,18,8.000000,264,33,Malaga,-1.553653,0.761880,-0.9372
4,La_liga,2014,h,0.278657,0.646364,0.278657,0.646364,4,7,0,...,9.451613,293,31,4.571429,96,21,Malaga,0.278657,0.646364,-0.1279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33369,RFPL,2022,h,1.073630,1.570220,1.073630,0.815948,7,7,3,...,9.550000,191,20,16.909091,186,11,Fakel,-1.926370,-1.429780,-0.0765
33370,RFPL,2022,a,0.889231,2.575200,0.134956,1.820930,2,16,1,...,7.931034,230,29,9.400000,141,15,Fakel,-0.110769,-1.424800,0.2661
33371,RFPL,2022,h,0.777784,0.852312,0.777784,0.852312,15,5,0,...,15.333333,230,15,8.052632,153,19,Fakel,0.777784,0.852312,0.2444
33372,RFPL,2022,a,0.642484,0.801640,0.642484,0.801640,8,3,1,...,10.458333,251,24,15.769231,205,13,Fakel,-0.357516,-0.198360,0.1997


In [ ]:
df.columns

Index(['league', 'year', 'h_a', 'xG', 'xGA', 'npxG', 'npxGA', 'deep',
       'deep_allowed', 'scored', 'missed', 'xpts', 'result', 'date', 'wins',
       'draws', 'loses', 'pts', 'npxGD', 'ppda_coef', 'ppda_att', 'ppda_def',
       'oppda_coef', 'oppda_att', 'oppda_def', 'team', 'xG_diff', 'xGA_diff',
       'xpts_diff'],
      dtype='object')

In [ ]:
cols = ['league','year','wins','loses','draws','pts','xG','xGA','npxGA','ppda_coef']

In [ ]:
df[cols]

,league,year,wins,loses,draws,pts,xG,xGA,npxGA,ppda_coef
0,La_liga,2014,1,0,0,3,1.321070,1.141510,1.141510,12.071429
1,La_liga,2014,0,1,0,0,1.131960,1.968630,1.069640,15.368421
2,La_liga,2014,0,0,1,1,0.819725,0.358265,0.358265,9.846154
3,La_liga,2014,0,0,1,1,0.446347,2.761880,2.761880,5.500000
4,La_liga,2014,0,0,1,1,0.278657,0.646364,0.646364,9.451613
...,...,...,...,...,...,...,...,...,...,...
33369,RFPL,2022,0,0,1,1,1.073630,1.570220,0.815948,9.550000
33370,RFPL,2022,0,1,0,0,0.889231,2.575200,1.820930,7.931034
33371,RFPL,2022,0,0,1,1,0.777784,0.852312,0.852312,15.333333
33372,RFPL,2022,0,0,1,1,0.642484,0.801640,0.801640,10.458333


In [ ]:
df = df[df['year'] == 2020]

In [ ]:
df = df[cols]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4132 entries, 4560 to 32781
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   league     4132 non-null   object 
 1   year       4132 non-null   int64  
 2   wins       4132 non-null   int64  
 3   loses      4132 non-null   int64  
 4   draws      4132 non-null   int64  
 5   pts        4132 non-null   int64  
 6   xG         4132 non-null   float64
 7   xGA        4132 non-null   float64
 8   npxGA      4132 non-null   float64
 9   ppda_coef  4132 non-null   float64
dtypes: float64(4), int64(5), object(1)
memory usage: 355.1+ KB


In [ ]:
sns.countplot(data=df,x='league')

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [ ]:
df

,league,year,wins,loses,draws,pts,xG,xGA,npxGA,ppda_coef
4560,La_liga,2020,1,0,0,3,2.210200,1.098300,1.098300,3.434783
4561,La_liga,2020,1,0,0,3,1.047290,0.375244,0.375244,6.500000
4562,La_liga,2020,0,0,1,1,1.255220,1.057170,1.057170,11.212121
4563,La_liga,2020,0,1,0,0,0.699011,1.087210,1.087210,11.411765
4564,La_liga,2020,0,1,0,0,1.985580,0.198368,0.198368,5.173913
...,...,...,...,...,...,...,...,...,...,...
32777,RFPL,2020,0,1,0,0,0.266019,2.789870,2.035600,14.846154
32778,RFPL,2020,0,1,0,0,0.023132,2.908110,2.908110,35.875000
32779,RFPL,2020,1,0,0,3,0.245349,0.870640,0.870640,9.826087
32780,RFPL,2020,0,1,0,0,1.203710,1.326340,1.326340,10.333333


In [ ]:
cols = ['pts','xG','xGA','npxGA','ppda_coef']
for i in cols:
  sns.histplot(df,x=i)
  plt.show()

ImportError: ignored

<Figure size 432x288 with 1 Axes>

ImportError: ignored

<Figure size 432x288 with 1 Axes>

ImportError: ignored

<Figure size 432x288 with 1 Axes>

ImportError: ignored

<Figure size 432x288 with 1 Axes>

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [ ]:
fig, ax = plt.subplots(figsize=(20,20))
sns.heatmap(df[cols].corr(), annot=False, linewidths=.5, ax=ax,cmap=sns.color_palette('Set2',as_cmap=True))

ImportError: ignored

<Figure size 1440x1440 with 2 Axes>

In [ ]:
values = df['league'].unique()

In [ ]:
df['wins_count'] = 0
for value in values:
  n_wins = len(df[df['league'] == value])
  index = df[df['league'] == value].index
  df.loc[index,'wins_count'] = n_wins

In [ ]:
x= df['league'].unique()
y = df['wins_count'].unique()

In [ ]:
x

array(['La_liga', 'EPL', 'Bundesliga', 'Serie_A', 'Ligue_1', 'RFPL'],
      dtype=object)

In [ ]:
y = [760,760,612,760,760,480]

In [ ]:
df[df['league']=='Ligue_1']

,league,year,wins,loses,draws,pts,xG,xGA,npxGA,ppda_coef,wins_count
27804,Ligue_1,2020,0,0,1,1,0.400974,1.295180,1.295180,16.500000,760
27805,Ligue_1,2020,1,0,0,3,0.906784,0.206627,0.206627,16.391304,760
27806,Ligue_1,2020,1,0,0,3,0.814528,0.736721,0.736721,6.029412,760
27807,Ligue_1,2020,0,0,1,1,1.494710,0.865098,0.865098,11.461538,760
27808,Ligue_1,2020,1,0,0,3,2.093870,0.660348,0.660348,8.000000,760
...,...,...,...,...,...,...,...,...,...,...,...
28559,Ligue_1,2020,1,0,0,3,0.920463,1.122260,1.122260,7.304348,760
28560,Ligue_1,2020,0,1,0,0,1.635170,2.618360,1.858260,7.800000,760
28561,Ligue_1,2020,0,1,0,0,1.026220,2.956970,2.956970,30.750000,760
28562,Ligue_1,2020,0,0,1,1,1.966960,0.408081,0.408081,11.105263,760


In [ ]:
x

array(['La_liga', 'EPL', 'Bundesliga', 'Serie_A', 'Ligue_1', 'RFPL'],
      dtype=object)

In [ ]:
y

[760, 760, 612, 760, 760, 480]

In [ ]:
sns.barplot(x=x,y=y)

ImportError: ignored

<Figure size 432x288 with 1 Axes>

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
cols = ['league','wins','pts','xG','xGA','ppda_coef']

In [ ]:
df = df[cols]

In [ ]:
df['league'].unique()

array(['La_liga', 'EPL', 'Bundesliga', 'Serie_A', 'Ligue_1', 'RFPL'],
      dtype=object)

In [ ]:
lbe = {'La_liga':0,'EPL':1,'Bundesliga':2,'Serie_A':3,'Ligue_1':4,'RFPL':5}

In [ ]:
df['league'] = df['league'].map(lbe)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
x_cols = ['league','pts','xG','xGA','ppda_coef']
x = df[x_cols]
y = df['wins']

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [ ]:
model = DecisionTreeClassifier()
model.fit(x_train,y_train)
feature_importance = np.array(model.feature_importances_)
feature_names = np.array(x_train.columns)
data={'feature_names':feature_names,'feature_importance':feature_importance}
df_plt = pd.DataFrame(data)
df_plt.sort_values(by=['feature_importance'], ascending=False,inplace=True)
plt.figure(figsize=(10,8))
sns.barplot(x=df_plt['feature_importance'], y=df_plt['feature_names'])
plt.xlabel('FEATURE IMPORTANCE')
plt.ylabel('FEATURE NAMES')
plt.show()

ImportError: ignored

<Figure size 720x576 with 1 Axes>

In [ ]:
preds = model.predict(x_test)
score = accuracy_score(preds,y_test)
print(score)

1.0


In [ ]:
df
x_test

,league,pts,xG,xGA,ppda_coef
16334,2,3,2.777480,0.664903,13.347826
11148,1,1,2.847080,0.860694,6.757576
16394,2,0,1.832220,2.085850,11.296296
16222,2,0,0.716727,2.075640,15.250000
11033,1,1,1.481500,2.260220,15.611111
...,...,...,...,...,...
10819,1,3,2.054500,0.579360,14.800000
4585,0,3,0.841992,0.662923,9.208333
32503,5,3,2.503600,0.368576,13.933333
21983,3,3,1.304510,2.498300,13.764706


In [ ]:
y_test

16334    1
11148    0
16394    0
16222    0
11033    0
        ..
10819    1
4585     1
32503    1
21983    1
11012    1
Name: wins, Length: 827, dtype: int64

In [ ]:
preds

array([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1,